In [75]:
import asyncio
from mpyc.runtime import mpc
from mpyc import asyncoro
import mpyc
import mpyc.gmpy
import numpy as np
import statsmodels.api as sm
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [76]:
import pandas as pd
iris = sm.datasets.get_rdataset("iris")
y = iris.data['Species']
X = iris.data.drop(columns=['Species'])
X = sm.add_constant(X)
X = X.to_numpy(dtype=np.float64)
y = pd.get_dummies(y, drop_first=True)["virginica"].to_numpy(dtype=np.float64)

In [77]:
import asyncio
from mpyc.runtime import mpc
from mpyc import asyncoro
import mpyc.gmpy

def factorial(x):
    res = 1
    for i in range(1,x+1):
        res = res*i
    return res

def exp(z):
    ftol = 1e-5
    exp_val = _exp(mpc.if_else(z < 0, -1, 1) * z, ftol)
    return mpc.if_else(z < 0, 1/exp_val, exp_val)

@asyncoro.mpc_coro
async def _exp(z, ftol):
    res = 1
    i = 1
    while True:
        term = (z**i)/factorial(i)
        res += term
        if await mpc.output(term < ftol): break
        i += 1
    return res

In [78]:
def log(x, q=3):
    ftol = 1e-5
    q = 4
    r = 0
    for _ in range(20):
        s = (x > q)
        x /= mpc.if_else(s, q, 1.0)
        r += 1 * s
    return _log(x, ftol) + r * 1.3862943611198906188

@asyncoro.mpc_coro
async def _log(x, ftol):
    ratio = (x-1)/(x+1)
    res = 0
    for i in range(1,100,2):
        term = 2*(ratio**i)/i
        res += term
        if await mpc.output(mpc.abs(term) < ftol): break
    return res


In [108]:
class SecurePoissonReg:

    def __init__(self, y, X):
        self.secfloat = mpc.SecFxp(128)
        if type(y) is np.ndarray:
            y = self.secfloat.array(y)
        if type(X) is np.ndarray:
            X = self.secfloat.array(X)
        self.y = y
        self.X = X
        self.n_obs = len(y)

    @asyncoro.mpc_coro
    async def fit(self, maxiter=1):
        beta = self.secfloat.array(np.zeros(self.X.shape[1]))
        for i in range(maxiter):
            score = self.score(beta)
            hessian = self.hessian(beta)
            beta += self.np_inv(hessian) @ score
            print(f"iter {i} completed: {await mpc.output(beta)}")
        return beta

    def likelihood(self):
        pass

    def score(self, beta):
        Xb = self.X @ beta
        self.S = self.apply_exp(Xb)
        return (self.y - self.S) @ self.X / self.n_obs

    def hessian(self, beta):
        return mpc.np_multiply(self.X.T, self.S) @ self.X / self.n_obs

    def apply_exp(self, X):
        X = X.copy()
        for i in range(len(X)):
            mpc.np_update(X, i, exp(X[i]))
        return X
    
    def np_inv(self, A, niter=100):
        n = A.shape[0]
        X_k = self.secfloat.array(np.eye(n)) / self.norm_1(A)
        for k in range(niter):
            X_k_next = 2 * X_k - X_k @ A @ X_k     
            X_k = X_k_next    
        return X_k_next
        
    def norm_1(self, A):
        n = mpc.np_sgn(A) * A
        n = mpc.np_sum(n, axis=0)
        n = mpc.np_amax(n)
        return n

In [109]:
model = SecurePoissonReg(y, X)
await mpc.output(model.fit(maxiter=9))

iter 0 completed: [-1.69528186 -0.04587608  0.20276839  0.00398791  0.55177932]
iter 1 completed: [-3.20934563 -0.12067269  0.30583192  0.18057823  0.88895736]
iter 2 completed: [-3.62438922 -0.26308894  0.09178759  0.48997542  1.03494321]
iter 3 completed: [-3.43254437 -0.47695683 -0.2659405   0.8477602   1.19067416]
iter 4 completed: [-3.36282657 -0.63695567 -0.45246954  1.07538864  1.31730339]
iter 5 completed: [-3.37492314 -0.67003569 -0.47652455  1.11814613  1.34415541]
iter 6 completed: [-3.3758207  -0.67096913 -0.47704604  1.11935514  1.34491814]
iter 7 completed: [-3.37582169 -0.67096987 -0.47704641  1.11935612  1.34491876]
iter 8 completed: [-3.37582169 -0.67096987 -0.47704641  1.11935612  1.34491876]


array([-3.37582169, -0.67096987, -0.47704641,  1.11935612,  1.34491876])

In [113]:
sm.Poisson(y, X).fit().params

Optimization terminated successfully.
         Current function value: 0.425146
         Iterations 8


array([-3.37582145, -0.67096978, -0.47704626,  1.11935597,  1.34491856])